In [1]:
import pyemu
import os
import pandas as pd
import matplotlib.pyplot as plt
import psutil

In [2]:
wd = "D:/spark/dolores_amrs"
os.chdir(wd)

In [3]:
# specify a template directory (i.e. the PstFrom working folder)
# tmp_d = os.path.join(wd, "Animas_AMRS-Fire_org")
template_ws = os.path.join(wd, "dolores_amrs")
start_datetime="1-1-1987"

In [4]:
pst_path = os.path.join(template_ws, 'dolores_amrs.pst')

In [5]:
# pst_path = os.path.join(t_d, 'freyberg_mf6.pst')
pst = pyemu.Pst(pst_path)

In [6]:
par = pst.parameter_data

In [7]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,...,pstyle,i,j,x,y,zone,usecol,idx0,idx1,idx2
parnme,,,,,,,,,,,,,,,,,,,,,
init_ca,init_ca,log,factor,90.0001,1.000000e-04,4.900000e+02,initc,1.0,-90.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
init_cl,init_cl,log,factor,90.0001,1.000000e-04,4.900000e+02,initc,1.0,-90.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
init_cno3,init_cno3,fixed,factor,90.0001,1.000000e-04,4.900000e+02,initc,1.0,-90.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
init_co3,init_co3,fixed,factor,90.0001,1.000000e-04,4.900000e+02,initc,1.0,-90.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
init_hco3,init_hco3,fixed,factor,90.0001,1.000000e-04,4.900000e+02,initc,1.0,-90.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dum,dum,log,factor,1.0000,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dum,dum,log,factor,1.0000,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dum,dum,log,factor,1.0000,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# pyemu.os_utils.run('pestpp-glm animas_amrs.pst', cwd=template_ws)

In [ ]:
pst.control_data.noptmax = -1

In [ ]:
pst.write(os.path.join(template_ws,"dolores_prior.pst"))

In [ ]:
pst.pestpp_options['ies_num_reals'] = 100
pst.control_data.noptmax = -1
pst.write(os.path.join(template_ws,"animas_diagprior2.pst"))

In [9]:
num_workers = psutil.cpu_count(logical=False)

In [8]:
m_d = "main_diagonal_prior_mc" 
pyemu.os_utils.start_workers(template_ws,"pestpp-ies","dolores_amrs_prior.pst",
                            num_workers=2,
                            worker_root='.',
                            master_dir=m_d)

C:\Users\seonggyu.park\AppData\Local\miniconda3\envs\apexmf\lib\site-packages\pyemu\utils\os_utils.py:143: PyemuWarning: unable to remove worker dir: .\worker_0
[WinError 5] Access is denied: '.\\worker_0\\pestpp-ies.exe'
C:\Users\seonggyu.park\AppData\Local\miniconda3\envs\apexmf\lib\site-packages\pyemu\utils\os_utils.py:143: PyemuWarning: unable to remove worker dir: .\worker_1
[WinError 5] Access is denied: '.\\worker_1\\pestpp-ies.exe'


Exception: start_workers() master returned non-zero: 3221226505

In [ ]:
pst = pyemu.Pst(os.path.join(template_ws, 'animas_diagprior.pst'))

In [ ]:
pst.write_par_summary_table(filename="none")

In [ ]:
pst.npar_adj

In [ ]:
obs_df = pd.read_csv(os.path.join(m_d,"animas_diagprior.0.obs.csv"),index_col=0)

In [ ]:
pst.try_parse_name_metadata()

In [ ]:
obs = pst.observation_data.copy()

In [ ]:
time_col = []
for i in range(len(obs)):
    if obs.iloc[i, 0][0] == "d":
        time_col.append(obs.iloc[i, 0][-8:])
    elif obs.iloc[i, 0][0] == "m":
        time_col.append(obs.iloc[i, 0][-6:])


In [ ]:
obs.time = time_col

In [ ]:
obs.loc[obs["obgnme"]=="m075_lso4"]

In [ ]:
ogs = obs.obgnme.unique()

In [ ]:
ogs

In [ ]:
ogs = [i for i in ogs if i[5] == "l"]

In [ ]:
fig,axes = plt.subplots(len(ogs),1,figsize=(10,5*len(ogs)))
ogs.sort()
for ax,og in zip(axes,ogs):
    oobs = obs.loc[obs.obgnme==og,:].copy()
    oobs.loc[:,"time"] = oobs.time.astype(str)
#     oobs.sort_values(by="time",inplace=True)
    tvals = oobs.time.values
    onames = oobs.obsnme.values
#     [ax.plot(tvals,obs_df.loc[i,onames].values,"0.5",lw=0.5,alpha=0.5) for i in obs_df.index]
    [ax.scatter(tvals,obs_df.loc[i,onames].values, color="gray", alpha=0.3) for i in obs_df.index]
    oobs = oobs.loc[oobs.weight>0,:]
    ax.scatter(oobs.time,oobs.obsval, color="red", label="Observed", s=30).set_facecolor("none")
    ax.tick_params(axis='x', labelrotation=45)
    ax.margins(x=0.01)
    ax.set_title(og,loc="left")